In [1]:
#%% Setup.

# Activate environment.
! . ~/env/bin/activate

# Install requirements.

! pip install --upgrade numpy torch scipy sklearn nibabel nilearn matplotlib

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 17.0MB 77kB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 676.9MB 2.1kB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 24.8MB 51kB/s 
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 3.3MB 435kB/s 
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 2.4MB 599kB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 12.8MB 85kB/s 
    100% |████████████████████████████████| 829kB 1.4MB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 5.5MB 187kB/s 
  Cach

## Imports

In [1]:
#%% Imports.

import numpy as np
import ardent

import matplotlib
%matplotlib inline

from pathlib import Path

/home/dcrowley/ARDENT_gpu_test/env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Prepare template and target

### Load template and target arrays from file.

Note: in this demo we use the Allen mouse brain atlas for the template, and thus the words 'atlas' and 'template' are effectively interchangable hereafter.

'atlas' is used in the code of this demo, whereas 'template' is used internally to ardent.

In [2]:
#%% Load template and target.

directory_path = Path('/home/dcrowley/image_lddmm_tensorflow')
atlas_image_filename = 'average_template_50.img'
target_image_filename = '180517_Downsample.img'

atlasPath = directory_path / atlas_image_filename
targetPath = directory_path / target_image_filename

atlas = ardent.load(atlasPath)
target = ardent.load(targetPath)

(160, 264, 228) (228, 160, 270)
(160, 264, 228) (228, 160, 270)


### Preprocess template and target 

In [ ]:
#%% Reform and normalize images.

atlas = ardent.basic_preprocessing(atlas)
target = ardent.basic_preprocessing(target)

# If you are wondering, the above does the same thing as below:
'''
# Normalize.
# Compute mean absolute deviation.
atlas_mean_absolute_deviation = np.mean(np.abs(atlas - np.median(atlas)))
target_mean_absolute_deviation = np.mean(np.abs(target - np.median(target)))
# Subtract mean.
atlas -= np.mean(atlas)
target -= np.mean(target)
# Divide by mean absolute deviation.
atlas /= atlas_mean_absolute_deviation
target /= target_mean_absolute_deviation

# Pad with zeros.
atlas = np.pad(atlas, pad_width=5, mode='constant', constant_values=0)
target = np.pad(target, pad_width=5, mode='constant', constant_values=0)
'''

## Orient target to template

### Visualize images

In [ ]:
#%% Visualize images before registration.
# It is important to check that the images are oriented to one another 
# and do not require rotating or flipping.

ardent.heatslices(atlas, title='atlas', limit_mode='stdev')
ardent.heatslices(target, title='target', limit_mode='stdev')

### Reorient target and revisualize

In [ ]:
#%% Orient images.
# From looking at the two images we can see that they are not oriented.
# This must be corrected prior to performing the registration.

# Here we will orient the target to the template (atlas).
reoriented_target = np.copy(target)
reoriented_target = np.rot90(reoriented_target, axes=(1,2))
reoriented_target = np.rot90(reoriented_target, axes=(2,0))
# np.moveaxis does nothing when destination == source.
# reoriented_target = np.moveaxis(reoriented_target, source=[0,1,2], destination=[0,1,2])
reoriented_target = np.flip(reoriented_target, axis=0)
reoriented_target = np.flip(reoriented_target, axis=1)

# If we performed an odd number of inversions with the above lines, 
# we must perform a flip to correct for inverting the image. 
# The simplest flip is a sagittal flip, since that is the axis of innate symmetry in the brain.
# In this case, we can see from the atlas visualization that this is axis 2 (the last row).
# The above lines performed 2 inversions, so the following line is commented out.
# reoriented_target = np.flip(reoriented_target, axis=2)

# Verify proper orientation.
ardent.heatslices(atlas, title='atlas', limit_mode='stdev')
ardent.heatslices(reoriented_target, title='reoriented target', limit_mode='stdev')

### Use reoriented target

In [ ]:
#%% Use reoriented target.
# Once we've finished orienting target, we use the result as target.
# A copy is taken to get a real array rather than a view.
target = reoriented_target.copy()

# Pad images to same shape.
atlas, target = ardent.lddmm.torch_lddmm_wrapper._pad_to_same_shape(atlas, target)

# Perform registration

In [ ]:
#%% Perform registration.

# Instantiate Transform object.
transform = ardent.Transform()

transform.register(template=atlas, target=target, 
    sigmaR=1e6, eV=0.1, niter=5)

# Apply transformation

In [ ]:
#%% Apply the transformation.
# The transformation is the result of the registration, and it can be applied to arbitrary images, 
# although presently they should be preprocessed as the atlas and target were.
# Here we will simply apply the transformation to both the atlas and target, 
# making them match up with each other.
# i.e. deformed_target should resemble atlas, and deformed_atlas should resemble target.

deformed_atlas = transform.apply_transform(subject=atlas, deform_to='target', save_path=None)
deformed_target = transform.apply_transform(subject=target, deform_to='template', save_path=None)

# Visualize results

In [ ]:
#%% Visualize results.

ardent.heatslices(deformed_atlas, title='deformed_atlas', limit_mode='stdev')
ardent.heatslices(deformed_target, title='deformed_target', limit_mode='stdev')

# To simply visualize error we plot the difference between images.
ardent.heatslices(deformed_target - atlas, title='error in atlas space', limit_mode='stdev')
ardent.heatslices(target - deformed_atlas, title='error in target space', limit_mode='stdev')
ardent.heatslices(target - atlas, title='error before registration', limit_mode='stdev')